In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from utils.df_loader import load_breast_cancer_df, load_diabetes_df
from sklearn.model_selection import train_test_split
from utils.preprocessing import preprocess_df
from utils.models import train_three_models, evaluation_test, save_three_models, load_three_models
from utils.watcher import  generate_watcher_result, process_result
from utils.watcher import Recorder
from utils.save import save_result_as_csv

### Disable TF2 and enable TF1 for alibi.
tf.get_logger().setLevel(40) 
tf.compat.v1.disable_v2_behavior()
tf.keras.backend.clear_session()
pd.options.mode.chained_assignment = None 

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False


seed = 123
# tf.random.set_seed(seed)
# np.random.seed(seed)


TF version:  2.4.0-rc0
Eager execution enabled:  False


In [2]:
#### Select dataset ####
dataset_name = 'diabetes' # [adult, german, compas]

if dataset_name == 'diabetes':
    dataset_loading_fn = load_diabetes_df
elif dataset_name == 'breast_cancer':
    dataset_loading_fn = load_breast_cancer_df
else:
    raise Exception("Unsupported dataset")

In [3]:
#### Load datafram info.
df_info = preprocess_df(dataset_loading_fn)

In [4]:
### Seperate to train and test set.
train_df, test_df = train_test_split(df_info.dummy_df, train_size=.8, random_state=seed, shuffle=True)

In [5]:
### Get training and testing array.
X_train = np.array(train_df[df_info.ohe_feature_names])
y_train = np.array(train_df[df_info.target_name])
X_test = np.array(test_df[df_info.ohe_feature_names])
y_test = np.array(test_df[df_info.target_name])

In [6]:
# ## Train models.
# models = train_three_models(X_train, y_train)

# ## Save models.
# save_three_models(models, dataset_name)

In [7]:
### Load models
models = load_three_models(X_train.shape[-1], dataset_name)

/Users/jrhs/miniforge3/envs/tf_mac/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [8]:
### Print out accuracy on testset.
evaluation_test(models, X_test, y_test)

DT: [0.7727] | RF [0.7987] | NN [0.7662]


## Watcher Counterfactual

In [9]:
### Setting up the CF generating amount.
num_instances = 5
num_cf_per_instance = 1

In [10]:
# Generate CF
results = generate_watcher_result(
    df_info,
    train_df,
    models,
    num_instances,
    num_cf_per_instance,
    X_train,
    X_test,
    y_test,
    max_iters=1000,
    models_to_run=['rfc', 'nn'],
    output_int=True,
)
result_dfs = process_result(results, df_info)

# Try not return float, return solid [0, 1]


Finding counterfactual for rfc
instance 0
CF 0
Found CF
instance 1
CF 0
CF not found
instance 2
CF 0
Found CF
instance 3
CF 0
CF not found
instance 4
CF 0
CF not found
Finding counterfactual for nn
instance 0
CF 0
CF not found
instance 1
CF 0
Found CF
instance 2
CF 0
Found CF
instance 3
CF 0
Found CF
instance 4
CF 0
Found CF


In [20]:
result_dfs['nn']['Found']

0    N
0    Y
0    Y
0    Y
0    Y
Name: Found, dtype: object

In [22]:
result_dfs['rfc']['Found']

0    Y
0    N
0    Y
0    N
0    N
Name: Found, dtype: object

In [11]:
i = 0
example_input = X_test[i:i+1]
Recorder.wrapped_models['nn'].predict(example_input)


array([[1]])

In [12]:
print(Recorder.wrapped_models['nn'].predict(example_input))
print(Recorder.wrapped_models['dt'].predict(example_input))
print(Recorder.wrapped_models['rfc'].predict(example_input))

[[1]]
[[1]]
[[1]]


In [13]:
print(Recorder.wrapped_models['nn'].predict_proba(example_input))
print(Recorder.wrapped_models['dt'].predict_proba(example_input))
print(Recorder.wrapped_models['rfc'].predict_proba(example_input))

[[0 1]]
[[0 1]]
[[0 1]]


In [19]:
### Save result as file.
save_result_as_csv("watcher", dataset_name, result_dfs)

Result has been saved to ./results/watcher_diabetes
